In [ ]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract
from ibapi.order import Order
import threading
import time

In [ ]:
class IBapi(EWrapper, EClient):
    def __init__(self):
        EWrapper.__init__(self)
        EClient.__init__(self, wrapper=self)

class GridTradingBot(IBapi):
    def __init__(self, grid_size, quantity):
        super().__init__()
        self.grid_size = grid_size
        self.quantity = quantity
        self.positions = []
        self.current_price = None  # Initialize current_price
        self.nextOrderId = None

    def start(self):
        self.connect("127.0.0.1", 7497, 1)
        thread = threading.Thread(target=self.run)
        thread.start()
        time.sleep(1)  # Sleep interval to allow time for connection to server

    def nextValidId(self, orderId: int):
        self.nextOrderId = orderId
        self.place_initial_orders()

    def place_initial_orders(self):
        # Place initial buy and sell orders
        current_price = self.get_current_price()
        if current_price is not None:
            buy_price = current_price - self.grid_size
            sell_price = current_price + self.grid_size
            self.place_order("BUY", self.quantity, buy_price)
            self.place_order("SELL", self.quantity, sell_price)

    def place_order(self, action, quantity, price):
        order = Order()  # Correctly initialize the Order object
        order.action = action
        order.totalQuantity = quantity
        order.orderType = "LMT"
        order.lmtPrice = price
        self.placeOrder(self.nextOrderId, self.create_contract(), order)
        self.nextOrderId += 1

    def create_contract(self):
        contract = Contract()  # Correctly initialize the Contract object
        contract.symbol = "MBT"  # Symbol for Micro Bitcoin Futures
        contract.secType = "FUT"  # Security type is futures
        contract.exchange = "CME"  # Exchange is CME
        contract.currency = "USD"  # Currency is USD
        contract.lastTradeDateOrContractMonth = "202312"  # Expiry date (e.g., December 2023)
        contract.multiplier = "0.1"  # Contract size (0.1 Bitcoin)
        print(f"Contract details: {contract.symbol}, {contract.secType}, {contract.exchange}, {contract.currency}, Expiry: {contract.lastTradeDateOrContractMonth}, Multiplier: {contract.multiplier}")
        return contract

    def get_current_price(self):
        contract = self.create_contract()
        self.reqMktData(1, contract, "", False, False, [])
        time.sleep(2)  # Sleep to allow time for price to be fetched
        print(f"Current price for Micro Bitcoin Futures: {self.current_price}")
        return self.current_price

    def tickPrice(self, reqId, tickType, price, attrib):
        if tickType == 4:  # Last price
            self.current_price = price
            print(f"TickPrice. ReqId: {reqId}, TickType: {tickType}, Price: {price}")

    def orderStatus(self, orderId, status, filled, remaining, avgFillPrice, permId, parentId, lastFillPrice, clientId, whyHeld, mktCapPrice):
        print(f"OrderStatus. Id: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, LastFillPrice: {lastFillPrice}")

    def execDetails(self, reqId, contract, execution):
        print(f"ExecDetails. ReqId: {reqId}, Symbol: {contract.symbol}, SecType: {contract.secType}, Currency: {contract.currency}, ExecId: {execution.execId}, OrderId: {execution.orderId}, Shares: {execution.shares}, LastLiquidity: {execution.lastLiquidity}")

# Example usage
bot = GridTradingBot(grid_size=10, quantity=1)  # Adjust grid_size and quantity as needed
bot_thread = threading.Thread(target=bot.start)
bot_thread.start()

# The bot will now run in the background, and you can continue using other cells in Jupyter Notebook.

Exception in thread Thread-5 (run):
Traceback (most recent call last):
  File "c:\Users\isaac\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\isaac\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\isaac\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\isaac\AppData\Local\Programs\Python\Python312\Lib\site-packages\ibapi\client.py", line 263, in run
    self.decoder.interpret(fields)
  File "c:\Users\isaac\AppData\Local\Programs\Python\Python312\Lib\site-packages\ibapi\decoder.py", line 1294, in interpret
    self.interpretWithSignature(fields, handleInfo)
  File "c:\Users\isaac\AppData\Local\Programs\Python\Python312\Lib\site-packages\ibapi\decoder.py", line 1275, in interpretWithSignature
    method(*args)
  File "C:\Users\isaac